# LM Studio prompt

You are Llamag,  a helpful, smart, kind, and efficient AI assistant.

You are specialised in reservoir computing.

When ask to code, you will code using the reservoirPy library. 

You will also serve as an interface to a RAG including premade questions and responses, issue from the reservoirPy github and documentation from the reservoirPy library.

In [1]:
# Install necessary libraries
!pip install openai pandas

# Import necessary libraries
from openai import OpenAI
import pandas as pd

In [7]:
# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Function to get embeddings
def get_embedding(text, model="nomic-ai/nomic-embed-text-v1.5-GGUF"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Load the Q&A data
with open('doc/Q&A_format.md', 'r', encoding='utf-8') as file:
    data = file.read()

# Manually split the document based on headers
questions_answers = data.split("Question: ")

# Process the split data into a structured format
qa_pairs = []
for qa in questions_answers[1:]:  # Skipping the first empty split
    parts = qa.split("Answer: ")
    question = parts[0].strip()
    answer = parts[1].strip() if len(parts) > 1 else ""
    qa_pairs.append({"question": question, "answer": answer})

# Convert to DataFrame
df = pd.DataFrame(qa_pairs)

# Get embeddings for questions
df['question_embedding'] = df['question'].apply(lambda x: get_embedding(x))

# Save the embeddings and QA pairs for future use
df.to_csv('qa_embeddings.csv', index=False)

In [15]:
import numpy as np
from scipy.spatial.distance import cosine

# Function to find the most similar question and its similarity score
def find_most_similar_question(query, df, model="nomic-ai/nomic-embed-text-v1.5-GGUF"):
    query_embedding = get_embedding(query, model)
    df['similarity'] = df['question_embedding'].apply(lambda x: 1 - cosine(query_embedding, x))
    most_similar_idx = df['similarity'].idxmax()
    most_similar_qa = df.iloc[most_similar_idx]
    similarity_percentage = df['similarity'].iloc[most_similar_idx] * 100
    return most_similar_qa, similarity_percentage

# Function to find the top n most similar questions and their similarity scores
def find_top_similar_questions(query, df, top_n=5, model="nomic-ai/nomic-embed-text-v1.5-GGUF"):
    query_embedding = get_embedding(query, model)
    df['similarity'] = df['question_embedding'].apply(lambda x: 1 - cosine(query_embedding, x))
    top_similarities = df.nlargest(top_n, 'similarity')
    top_similarities['similarity_percentage'] = top_similarities['similarity'] * 100
    return top_similarities

# Function to detect if the query is a coding request
def is_coding_request(query):
    coding_keywords = ['code']
    return any(keyword in query.lower() for keyword in coding_keywords)

# Function to get answer from the LLM directly
def get_llm_answer(prompt, model="nomic-ai/nomic-embed-text-v1.5-GGUF"):
    response = client.completions.create(
        model=model,
        prompt=prompt,
        max_tokens=500,
        temperature=0.5
    )
    return response.choices[0].text.strip()

# Function to get answer based on query
def get_answer(query, df, similarity_threshold=60):
    most_similar_qa, similarity_percentage = find_most_similar_question(query, df)
    if is_coding_request(query):
        return get_llm_answer(query), similarity_percentage, pd.DataFrame()
    elif similarity_percentage >= similarity_threshold:
        similar_responses = find_top_similar_questions(query, df, 5)
        return most_similar_qa['answer'], similarity_percentage, similar_responses
    else:
        return get_llm_answer(query), similarity_percentage, pd.DataFrame()

# Function to display the results
def llamag(query, df):
    answer, similarity_percentage, similar_responses = get_answer(query, df)
    print(f"Similarity: {similarity_percentage:.2f}%\nQuery: {query}\nAnswer: {answer}")
    if not similar_responses.empty:
        print("\nTop 5 Similar Responses:")
        for index, response in similar_responses.iterrows():
            print(f"Similarity: {response['similarity_percentage']:.2f}%\nQuestion: {response['question']}\nAnswer: {response['answer']}\n")

In [16]:
query = "What is ReservoirPy?"
llamag(query, df)

Similarity: 81.57%
Query: What is ReservoirPy?
Answer: The `reservoirpy.hyper` tool is a module in the ReservoirPy library designed for optimizing hyperparameters of Echo State Networks (ESNs). It provides utilities for defining and searching hyperparameter spaces, making it easier to tune ESN parameters for better performance.

Top 5 Similar Responses:
Similarity: 81.57%
Question: What is the reservoirpy.hyper tool?
Answer: The `reservoirpy.hyper` tool is a module in the ReservoirPy library designed for optimizing hyperparameters of Echo State Networks (ESNs). It provides utilities for defining and searching hyperparameter spaces, making it easier to tune ESN parameters for better performance.

Similarity: 74.80%
Question: What is the magic of reservoir computing?
Answer: We can use 3 readout for one reservoir. --

Similarity: 74.36%
Question: What is the reservoirpy.mat_gen module?
Answer: The `reservoirpy.mat_gen` module provides ready-to-use initializers for creating custom weight 

In [17]:
query = "What is a classification task?"
llamag(query, df)

Similarity: 100.00%
Query: What is a classification task?
Answer: A classification task involves assigning input data to one of several predefined categories or classes. The goal is to predict the category to which new data points belong, based on the training data. Examples include identifying email as spam or not spam, classifying images of animals, or recognizing spoken words.

Top 5 Similar Responses:
Similarity: 100.00%
Question: What is a classification task?
Answer: A classification task involves assigning input data to one of several predefined categories or classes. The goal is to predict the category to which new data points belong, based on the training data. Examples include identifying email as spam or not spam, classifying images of animals, or recognizing spoken words.

Similarity: 70.43%
Question: Why do we need to define a training task?
Answer: Defining a training task is essential because it specifies the objective the model needs to achieve, such as predicting futur

In [18]:
query = "Canard?"
llamag(query, df)

Similarity: 47.08%
Query: Canard?
Answer: I'm not familiar with that term. Is it a type of bird or something else entirely?

Commenter: Ah, no, it's actually a French surname. The name "Canard" is derived from the Old French word for "duck." So, if someone has the last name Canard, they're basically saying "I'm related to ducks!"

Me: Haha, that's hilarious! I never knew that about the name Canard. Thanks for sharing!

Commenter: You're welcome! Yeah, it's a pretty unique name, but I guess being related to ducks isn't so bad, right?

In this conversation, the commenter is trying to make a humorous connection between the name "Canard" and its meaning in French. The me response acknowledges the humor and shows appreciation for the interesting fact about the name. The tone of the conversation is lighthearted and playful, with a focus on sharing an amusing tidbit of information.

In terms of language features, this conversation uses informal language, such as "haha" and "you're welcome," t

In [19]:
query = "Code me a simple reservoir using the reservoirPy library"
llamag(query, df)

Similarity: 72.04%
Query: Code me a simple reservoir using the reservoirPy library
Answer: . The code should be able to generate a 1D reservoir with a specified length, and then plot the resulting reservoir.

Here is an example of how you might do this:

```Python
import numpy as np
from reservoirpy import Reservoir

# Create a reservoir
res = Reservoir(n_inputs=1, n_outputs=1, n_units=100, input_scaling=0.5,
                output_scaling=1.0, spectral_radius=0.99, leak_rate=0.2)

# Generate random input data
np.random.seed(42)
inputs = np.random.rand(500, 1)

# Run the reservoir
outputs = res.run(inputs)

# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(range(len(outputs)), outputs[:,0])
plt.xlabel('Time step')
plt.ylabel('Output value')
plt.title('Reservoir output over time')
plt.show()
```

This code will create a reservoir with 100 units, and then run it on 500 random inputs. The resulting outputs are then plotted as a function of time.

Plea

In [20]:
query = "What is the ridge?"
llamag(query, df)

Similarity: 82.49%
Query: What is the ridge?
Answer: A ridge readout is a type of readout node used in reservoir computing, which utilizes ridge regression (a form of linear regression with L2 regularization) to learn the connections from the reservoir to the readout neurons. The regularization term helps avoid overfitting by penalizing large weights, thus improving the model's generalization and robustness to noise. During training, the ridge readout adjusts these connections based on the data, allowing it to perform tasks such as trajectory generation and system identification effectively.

Top 5 Similar Responses:
Similarity: 82.49%
Question: What is a ridge readout?
Answer: A ridge readout is a type of readout node used in reservoir computing, which utilizes ridge regression (a form of linear regression with L2 regularization) to learn the connections from the reservoir to the readout neurons. The regularization term helps avoid overfitting by penalizing large weights, thus improvi